In [ ]:
import asyncio
from pynq import GPIO
from pynq import Overlay
overlay = Overlay("block_design_top.bit")

In [ ]:
opcode_bits = list(range(51,48,-1))
done_bit = 48
out_bits = list(range(47,23,-1))
in2_bits = list(range(23,15,-1))
in1_bits = list(range(15,7,-1))
in0_bits = list(range(7,-1,-1))

In [ ]:
def set_output(pin_number):
    return GPIO(GPIO.get_gpio_pin(pin_number), 'out')

def set_input(pin_number):
    return GPIO(GPIO.get_gpio_pin(pin_number), 'in')

opcode = list(map(set_output, opcode_bits))
done = set_input(done_bit)
out = list(map(set_input, out_bits))
in2 = list(map(set_output, in2_bits))
in1 = list(map(set_output, in1_bits))
in0 = list(map(set_output, in0_bits))

In [ ]:
def wait_for_result():
    while True: # spin until done. Most definitely unnecessary but awkward if needed and not there
        if done.read() == 1:
            break
    def reader(pin):
        return pin.read()
    result = 0;
    for i in range(len(out)):
        result += out[i].read() << (len(out) - i - 1)
    return result

def setup_input(input_number, value):
    if input_number == 2:
        inp = in2
    elif input_number == 1:
        inp = in1
    elif input_number == 0:
        inp = in0
    for i in range(8):
        inp[i].write((value >> (7 - i)) & 1)

def setup_opcode(opcode_value):
    for i in range(len(opcode)):
        opcode[i].write((opcode_value >> (len(opcode) - i - 1)) & 1)
    return

def operator_body1(a):
    setup_input(0, a)
    return wait_for_result()

def operator_body2(a, b):
    setup_input(1, a)
    return operator_body1(b)

def operator_body3(a, b, c):
    setup_input(2, a)
    return operator_body2(b, c)

def operator_body(a, b = None, c = None):
    if b is not None and c is not None:
        return operator_body3(a, b, c)
    elif b is not None and c is None:
        return operator_body2(a, b)
    else:
        return operator_body1(a)

In [ ]:
def add(a, b):
    setup_opcode(0)
    return operator_body(a, b)

def sub(a, b):
    setup_opcode(1)
    return operator_body(a, b)

def mult(a, b):
    setup_opcode(2)
    return operator_body(a, b)

def shift_right(a, b):
    setup_opcode(3)
    return operator_body(a, b)

def square(a):
    setup_opcode(4)
    return operator_body(a)

def cube(a):
    setup_opcode(5)
    return operator_body(a)

def triple_add(a, b, c):
    setup_opcode(6)
    return operator_body(a, b, c)

def f(a, b, c):
    setup_opcode(7)
    return operator_body(a, b, c)

In [ ]:
print(f(0,0,0))